In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import json
import ast
from tqdm import tqdm
import requests
import warnings
warnings.filterwarnings('ignore') # turn off warnings

import psycopg2

connUAT = psycopg2.connect(
     host='localhost',
     database='uat',
     port=5431,
     user='postgres',
     password='1234567890'
  )

connPROC = psycopg2.connect(
     host='localhost',
     database='prod',
     port=5431,
     user='postgres',
     password='1234567890'
  )

cursorUAT = connUAT.cursor()
cursorPROC = connPROC.cursor()

def selectdf(query,conn):
    df = pd.read_sql(query, conn)
    return df
  
def SQL_push(sql_table_name,python_table,con,cursor,inplace):
    python_table.replace([np.nan], [None],inplace=True)
    
    if inplace == True:
        try:
            sql_query = f'truncate table {sql_table_name}'
            cursorUAT.execute(sql_query)
            connUAT.commit()
        except:
            pass
            
    sql = f"INSERT INTO {sql_table_name} VALUES ({','.join(['%s'] * len(python_table.columns))})"

    data = [tuple(row) for _, row in tqdm(python_table.iterrows(), total=len(python_table), desc="Preparing data")]

    # Chèn dữ liệu với xử lý lỗi
    for row in tqdm(data, total=len(data), desc="Inserting rows"):
        try:
            cursor.execute(sql, row)
        except Exception as e:
            print(f"Lỗi khi chèn dòng {row}: {e}")
            con.rollback()  # Hủy giao dịch khi có lỗi, tránh PostgreSQL khóa transaction

    # Sau khi xong thì commit lại
    connUAT.commit()
    print("PUSH DATA: DONE")

pathfolder = r'F:\Work\Caohoc_2024_2026\PTPM_project\kaggle_data'

In [ ]:
sql_check = '''
            select *
            from staging.stg_tmdb_json_movie_metadata
            '''
movie_json = selectdf(sql_check,connUAT)
# list_c['id'] = list_c['id'].astype(str)  # Đảm bảo cùng kiểu dữ liệu
movie_json.sort_values(by='id',inplace=True)
movie_json

,id,request_json
86196,2,"{""adult"": false, ""backdrop_path"": ""/hQ4pYsIbP2..."
86160,3,"{""adult"": false, ""backdrop_path"": ""/l94l89eMmF..."
86161,5,"{""adult"": false, ""backdrop_path"": ""/f2t4JbUvQI..."
86162,6,"{""adult"": false, ""backdrop_path"": ""/iUUpKunmBN..."
86163,11,"{""adult"": false, ""backdrop_path"": ""/2w4xG178Rp..."
...,...,...
70144,1144338,"{""adult"": false, ""backdrop_path"": null, ""belon..."
86156,1145358,"{""adult"": false, ""backdrop_path"": ""/4OIJmgez12..."
86157,1146246,"{""adult"": false, ""backdrop_path"": ""/t86SkVd35W..."
86158,1146249,"{""adult"": false, ""backdrop_path"": ""/eNopYAzvtf..."


In [63]:
data_str = movie_json[movie_json['id']==524288]['request_json'].values[0]
data = f'''{data_str}'''
data_dict = json.loads(data)
data_dict

{'adult': False,
 'backdrop_path': '/9uW5X9fdtfJp0bSW60Eqh0lqcEt.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 99, 'name': 'Documentary'}],
 'homepage': 'https://www.netflix.com/title/81074663',
 'id': 524288,
 'imdb_id': 'tt6939026',
 'origin_country': ['IN'],
 'original_language': 'hi',
 'original_title': 'Period. End of Sentence.',
 'overview': 'In an effort to improve feminine hygiene, a machine that creates low-cost biodegradable sanitary pads is installed in a rural village in Northern India. Using the machine, a group of local women is employed to produce and sell pads, offering them newfound independence and helping to destigmatize menstruation for all.',
 'popularity': 0.6451,
 'poster_path': '/dsCeBj8oabzsHQOGGLPrmrqIvDs.jpg',
 'production_companies': [{'id': 22486,
   'logo_path': '/a101r8HZ9dk6xvXyFHcEyh73Qpr.png',
   'name': 'Sikhya Entertainment',
   'origin_country': 'IN'},
  {'id': 115551,
   'logo_path': None,
   'name': 'The Pad Project',
   '

In [ ]:
data_dict_list = []

for i in tqdm(set(movie_json['id'])):
    data_str = movie_json[movie_json['id']==i]['request_json'].values[0]
    data = f'''{data_str}'''
    data_dict = json.loads(data)
    data_dict_list.append(data_dict)
    
movie_metadata = pd.DataFrame(data_dict_list)
movie_metadata

100%|██████████| 86374/86374 [00:28<00:00, 3043.92it/s]


[{'adult': False,
  'backdrop_path': '/9uW5X9fdtfJp0bSW60Eqh0lqcEt.jpg',
  'belongs_to_collection': None,
  'budget': 0,
  'genres': [{'id': 99, 'name': 'Documentary'}],
  'homepage': 'https://www.netflix.com/title/81074663',
  'id': 524288,
  'imdb_id': 'tt6939026',
  'origin_country': ['IN'],
  'original_language': 'hi',
  'original_title': 'Period. End of Sentence.',
  'overview': 'In an effort to improve feminine hygiene, a machine that creates low-cost biodegradable sanitary pads is installed in a rural village in Northern India. Using the machine, a group of local women is employed to produce and sell pads, offering them newfound independence and helping to destigmatize menstruation for all.',
  'popularity': 0.6451,
  'poster_path': '/dsCeBj8oabzsHQOGGLPrmrqIvDs.jpg',
  'production_companies': [{'id': 22486,
    'logo_path': '/a101r8HZ9dk6xvXyFHcEyh73Qpr.png',
    'name': 'Sikhya Entertainment',
    'origin_country': 'IN'},
   {'id': 115551,
    'logo_path': None,
    'name': 'T

In [62]:
pd.DataFrame(data_dict_list)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,success,status_code,status_message
0,False,/9uW5X9fdtfJp0bSW60Eqh0lqcEt.jpg,None,0.0,"[{'id': 99, 'name': 'Documentary'}]",https://www.netflix.com/title/81074663,524288.0,tt6939026,[IN],hi,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Period. End of Sentence.,False,7.800,238.0,NaN,NaN,NaN
1,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,2.0,tt0094675,[FI],fi,...,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,,Ariel,False,7.129,345.0,NaN,NaN,NaN
2,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,"{'id': 1382526, 'name': 'Aki Kaurismäki's Prol...",0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,3.0,tt0092149,[FI],fi,...,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,,Shadows in Paradise,False,7.306,407.0,NaN,NaN,NaN
3,False,/f2t4JbUvQIjUF5FstG1zZFAp02N.jpg,None,4000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",https://www.miramax.com/movie/four-rooms/,5.0,tt0113101,[US],en,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.900,2683.0,NaN,NaN,NaN
4,False,/iUUpKunmBN5l8goObADBaFHnxQ8.jpg,None,21000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",,6.0,tt0107286,[US],en,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.500,346.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86369,False,/d0LvPveoC0Tu2CrvoexOgFcp38F.jpg,None,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",,262120.0,tt0255713,[IN],hi,...,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,,Zubeidaa,False,5.200,14.0,NaN,NaN,NaN
86370,False,/2Pbjc94lE67VoD9JJluQJAtQ5Ni.jpg,None,0.0,"[{'id': 80, 'name': 'Crime'}, {'id': 12, 'name...",,262123.0,tt0032533,[US],en,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,TOUGH HIDES...and STOUT HEARTS!...THE DEAD END...,Give Us Wings,False,6.000,2.0,NaN,NaN,NaN
86371,False,/uQ9ZK4mo6ZWbNjVOc95lSzKWHFx.jpg,None,0.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",,786415.0,tt11348164,[US],en,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Iké Boys,False,5.800,9.0,NaN,NaN,NaN
86372,False,None,None,0.0,"[{'id': 10749, 'name': 'Romance'}]",,524275.0,tt3715208,[US],en,...,[],Released,,Love in Tokyo,False,6.000,2.0,NaN,NaN,NaN


In [ ]:
data_dict_list = []

data_str = movie_json[movie_json['id']==2]['request_json'].values[0]
data = f'''{data_str}'''
data_dict = json.loads(data)
data_dict

{'adult': False,
 'backdrop_path': '/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg',
 'belongs_to_collection': {'id': 1382526,
  'name': "Aki Kaurismäki's Proletariat Trilogy",
  'poster_path': '/bUrReoZFLGti6ehkBW0xw8f12MT.jpg',
  'backdrop_path': '/zAUItK1Nr473DIe8gWMsZ0DMR7L.jpg'},
 'budget': 0,
 'genres': [{'id': 35, 'name': 'Comedy'},
  {'id': 18, 'name': 'Drama'},
  {'id': 10749, 'name': 'Romance'},
  {'id': 80, 'name': 'Crime'}],
 'homepage': '',
 'id': 2,
 'imdb_id': 'tt0094675',
 'origin_country': ['FI'],
 'original_language': 'fi',
 'original_title': 'Ariel',
 'overview': 'A Finnish man goes to the city to find a job after the mine where he worked is closed and his father commits suicide.',
 'popularity': 1.5038,
 'poster_path': '/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg',
 'production_companies': [{'id': 2303,
   'logo_path': None,
   'name': 'Villealfa Filmproductions',
   'origin_country': 'FI'}],
 'production_countries': [{'iso_3166_1': 'FI', 'name': 'Finland'}],
 'release_date': '1988-10-21',


In [11]:
filename = 'movies_metadata' #  index = 35587
md = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
md = md.drop([19730, 29503, 35587])
md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [43]:
md_data_col = md[['id','imdb_id','title','original_title','original_language','release_date','status','overview','tagline','adult','popularity','homepage','poster_path','runtime','budget','revenue','vote_average','vote_count']]
md_data_col.head(5)

,id,imdb_id,title,original_title,original_language,release_date,status,overview,tagline,adult,popularity,homepage,poster_path,runtime,budget,revenue,vote_average,vote_count
0,862,tt0114709,Toy Story,Toy Story,en,1995-10-30,Released,"Led by Woody, Andy's toys live happily in his ...",NaN,False,21.946943,http://toystory.disney.com/toy-story,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,81.0,30000000,373554033.0,7.7,5415.0
1,8844,tt0113497,Jumanji,Jumanji,en,1995-12-15,Released,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,False,17.015539,NaN,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,104.0,65000000,262797249.0,6.9,2413.0
2,15602,tt0113228,Grumpier Old Men,Grumpier Old Men,en,1995-12-22,Released,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,False,11.7129,NaN,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,101.0,0,0.0,6.5,92.0
3,31357,tt0114885,Waiting to Exhale,Waiting to Exhale,en,1995-12-22,Released,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,False,3.859495,NaN,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,127.0,16000000,81452156.0,6.1,34.0
4,11862,tt0113041,Father of the Bride Part II,Father of the Bride Part II,en,1995-02-10,Released,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,False,8.387519,NaN,/e64sOI48hQXyru7naBFyssKFxVd.jpg,106.0,0,76578911.0,5.7,173.0


In [24]:
md.columns.to_list()

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [25]:
md[['id','production_companies']]

,id,production_companies
0,862,"[{'name': 'Pixar Animation Studios', 'id': 3}]"
1,8844,"[{'name': 'TriStar Pictures', 'id': 559}, {'na..."
2,15602,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'..."
3,31357,[{'name': 'Twentieth Century Fox Film Corporat...
4,11862,"[{'name': 'Sandollar Productions', 'id': 5842}..."
...,...,...
45461,439050,[]
45462,111109,"[{'name': 'Sine Olivia', 'id': 19653}]"
45463,67758,"[{'name': 'American World Pictures', 'id': 6165}]"
45464,227506,"[{'name': 'Yermoliev', 'id': 88753}]"


In [53]:
md[['id','production_companies']]['production_companies'][4]

"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]"

In [26]:
md[['id','production_countries']]

,id,production_countries
0,862,"[{'iso_3166_1': 'US', 'name': 'United States o..."
1,8844,"[{'iso_3166_1': 'US', 'name': 'United States o..."
2,15602,"[{'iso_3166_1': 'US', 'name': 'United States o..."
3,31357,"[{'iso_3166_1': 'US', 'name': 'United States o..."
4,11862,"[{'iso_3166_1': 'US', 'name': 'United States o..."
...,...,...
45461,439050,"[{'iso_3166_1': 'IR', 'name': 'Iran'}]"
45462,111109,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]"
45463,67758,"[{'iso_3166_1': 'US', 'name': 'United States o..."
45464,227506,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]"


In [31]:
md[['id','production_countries']]['production_countries'][0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [23]:
belongs_to_collection_raw = md[['id','belongs_to_collection']]
# belongs_to_collection = belongs_to_collection_raw['belongs_to_collection'][0]
# belongs_to_collection
belongs_to_collection_raw

,id,belongs_to_collection
0,862,"{'id': 10194, 'name': 'Toy Story Collection', ..."
1,8844,NaN
2,15602,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
3,31357,NaN
4,11862,"{'id': 96871, 'name': 'Father of the Bride Col..."
...,...,...
45461,439050,NaN
45462,111109,NaN
45463,67758,NaN
45464,227506,NaN


In [21]:
genres_raw = md[['id','genres']]
# genres = genres_raw['genres'][0]
# genres
genres_raw

,id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...
45461,439050,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
45462,111109,"[{'id': 18, 'name': 'Drama'}]"
45463,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,[]


In [51]:
data_dict = ast.literal_eval(genres_raw['genres'][0])
df = pd.DataFrame(data_dict)
df

,id,name
0,16,Animation
1,35,Comedy
2,10751,Family


In [22]:
spoken_languages_raw = md[['id','spoken_languages']]
# genres = genres_raw['genres'][0]
# genres
spoken_languages_raw

,id,spoken_languages
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,15602,"[{'iso_639_1': 'en', 'name': 'English'}]"
3,31357,"[{'iso_639_1': 'en', 'name': 'English'}]"
4,11862,"[{'iso_639_1': 'en', 'name': 'English'}]"
...,...,...
45461,439050,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]"
45462,111109,"[{'iso_639_1': 'tl', 'name': ''}]"
45463,67758,"[{'iso_639_1': 'en', 'name': 'English'}]"
45464,227506,[]


In [ ]:
filename = 'credits'
credits = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
credits

In [ ]:
credits_cast = credits[['id','cast']]
credits_cast

In [ ]:
ast.literal_eval(credits[credits['id']==862]['crew'].values[0])

In [14]:
df_cast = pd.DataFrame(ast.literal_eval(credits[credits['id']==862]['cast'].values[0]))
df_cast


,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
5,19,Hamm (voice),52fe4284c3a36847f8024fa9,2,7907,John Ratzenberger,5,/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg
6,20,Bo Peep (voice),52fe4284c3a36847f8024fad,1,8873,Annie Potts,6,/eryXT84RL41jHSJcMy4kS3u9y6w.jpg
7,26,Andy (voice),52fe4284c3a36847f8024fc1,0,1116442,John Morris,7,/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg
8,22,Sid (voice),52fe4284c3a36847f8024fb1,2,12901,Erik von Detten,8,/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg
9,23,Mrs. Davis (voice),52fe4284c3a36847f8024fb5,1,12133,Laurie Metcalf,9,/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg


In [10]:
# data_str = smd[smd.id==863]['keywords'].values[0]
# a = ast.literal_eval(data_str)
# a[0]['name']

In [ ]:
# filename = 'keywords'
# keywords = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
# keywords

In [ ]:
# ast.literal_eval(keywords[keywords['id']==11]['keywords'].values[0])
# data_json = keywords[keywords['id']==862]['keywords']
# data_json
# # json.loads(data_json.values[0])

In [7]:
# filename = 'ratings'
# ratings = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
# ratings

In [8]:
# filename = 'links'
# links = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
# links

In [9]:
# filename = 'links_small'
# links_small = pd.read_csv(os.path.join(pathfolder,(filename+'.csv')))
# links_small